<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/024_ParseResponse_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Planning + Parsing

### 1. **LLMs don't execute — they plan.**

* The LLM returns a **structured action plan**, not code or results.
* Your Python code interprets and executes those steps.

---

### 2. **Structured outputs are key to reliability.**

* Markdown code blocks (like ` ```action `) and strict JSON formats make parsing easy.
* Avoiding prose ensures your agent can work **without guesswork**.

---

### 3. **You are building a “tool-using brain.”**

* LLMs decide which tools to use, and with what args.
* Your code is the executor — like giving the brain a body.

---

### 4. **Few-shot examples dramatically improve results.**

* Showing the LLM *how* to respond increases consistency.
* Define your tool schema + show examples in your system prompt.

---

### 5. **Agents can scale up quickly — and costs can too.**

* One LLM call per plan = affordable.
* One LLM call per file = 🧨 60K tokens/day if you’re not careful.
* Plan your architecture with batching, reuse, and simplicity in mind.

---

### 6. **Simulated tools are the best way to learn.**

* You don’t need real summaries or file moves at first.
* Simulate → debug → validate → then implement.







## 🧠 **What’s the next level of parsing?**

### ➕ Add complexity to the return structure:

* Multiple actions
* Conditional steps
* Multi-tool reasoning

This simulates real-world agent orchestration.

---

## 🔥 Recommended Next Step:

# **🧩 Multi-Tool Planner Agent**

---

### 🧠 Agent Goal:

You give a natural language task like:

> “Summarize each file in docs\_folder and then move it into a topic folder.”

The LLM returns **a list of structured steps**, like:

````markdown
```action
[
  {
    "tool_name": "summarize_file",
    "args": { "filename": "lecture_01.txt" }
  },
  {
    "tool_name": "move_file",
    "args": {
      "filename": "lecture_01.txt",
      "target_folder": "Memory_Management"
    }
  }
]
````



### ✅ What you’ll build:
- LLM returns a list of actions (instead of one)
- You parse it with `json.loads()` into a **list of dicts**
- You loop through each step and execute the corresponding tool (summarize, move, rename, tag…)

---

## 💡 What this teaches you:

| Concept | Why it matters |
|--------|----------------|
| ✅ Multi-step reasoning | Required for real planning agents |
| ✅ Tool orchestration | Foundation of LangChain / OpenAgents style design |
| ✅ Dynamic routing | Agent decides what function to run |
| ✅ Multi-action parsing | Real-world LLMs often return batches of actions |

---

## 🛠️ Tools You'll Simulate

| Tool | Args |
|------|------|
| `summarize_file` | `filename: str` |
| `move_file` | `filename: str`, `target_folder: str` |
| `tag_file` (optional) | `filename: str`, `tag: str` |




In [ ]:
%pip install -qU dotenv openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 20.9 MB/s eta 0:00:00


## Python Coded Actions

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import re

load_dotenv("/content/API_KEYS.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

source_dir = "/content/docs_folder"

# Make sure the directory exists
if not os.path.exists(source_dir):
    raise FileNotFoundError(f"📁 Directory not found: {source_dir}")

# List and build full file paths
file_list = [
    os.path.join(source_dir, f)
    for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
]

# Display the found files
print("📂 Files found:")
for file in file_list:
    print("  -", file)

📂 Files found:
  - /content/docs_folder/001_PArse_the Response.txt
  - /content/docs_folder/006_Agent Loop with Function Calling.txt
  - /content/docs_folder/000_Prompting for Agents -GAIL.txt
  - /content/docs_folder/002_Execute_the_Action.txt
  - /content/docs_folder/005_Using Function Calling Capabilities with LLMs.txt
  - /content/docs_folder/003_gent Feedback and Memory.txt
  - /content/docs_folder/004_AGENT_Tools.txt


## LLM Coded Actions


In [ ]:
# build the multi action prompt
def build_multi_action_prompt(file_list):
    file_names = [os.path.basename(f) for f in file_list]
    file_list_str = "\n".join(f"- {name}" for name in file_names)

    system_prompt = (
        "You are an agent planner that creates a list of tool-based actions.\n"
        "You always respond with a JSON list of tool invocations inside a markdown code block, like this:\n\n"
        "```action\n"
        "[\n"
        "  {\n"
        "    \"tool_name\": \"summarize_file\",\n"
        "    \"args\": { \"filename\": \"lecture_01.txt\" }\n"
        "  },\n"
        "  {\n"
        "    \"tool_name\": \"move_file\",\n"
        "    \"args\": { \"filename\": \"lecture_01.txt\", \"target_folder\": \"Memory_Management\" }\n"
        "  }\n"
        "]\n"
        "```\n\n"
        "Available tools:\n"
        "- summarize_file: takes {\"filename\": str}\n"
        "- move_file: takes {\"filename\": str, \"target_folder\": str}"
    )

    user_prompt = (
        f"I want to summarize and organize the following files:\n\n{file_list_str}\n\n"
        "Please create an ordered list of tool actions that I should perform."
    )

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

def generate_response(messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=700
    )
    return response.choices[0].message.content

messages = build_multi_action_prompt(file_list)
llm_output = generate_response(messages)

print("🤖 LLM Response:\n")
print(llm_output)


# 📦 Parse the Markdown Block
def extract_markdown_block(text: str, tag: str = "action") -> str:
    pattern = rf"```{tag}\s*(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError(f"Missing markdown block with tag '{tag}'")

# ✅ Load it into a Python list
def parse_multi_action_plan(llm_response):
    try:
        json_block = extract_markdown_block(llm_response, tag="action")
        return json.loads(json_block)
    except Exception as e:
        print("❌ Failed to parse multi-action plan:", e)
        return []

# Agent Executes Each Action
def simulate_tool(tool_name, args):
    if tool_name == "summarize_file":
        print(f"📝 Summarizing: {args['filename']}")
    elif tool_name == "move_file":
        print(f"📂 Moving '{args['filename']}' → '{args['target_folder']}'")
    else:
        print(f"⚠️ Unknown tool: {tool_name}")

plan = parse_multi_action_plan(generate_response(build_multi_action_prompt(file_list)))

print("\n🛠️ Executing agent plan...\n")
for step in plan:
    simulate_tool(step["tool_name"], step["args"])


🤖 LLM Response:

```action
[
  {
    "tool_name": "summarize_file",
    "args": { "filename": "000_Prompting for Agents -GAIL.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "001_PArse_the Response.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "002_Execute_the_Action.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "003_gent Feedback and Memory.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "004_AGENT_Tools.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "005_Using Function Calling Capabilities with LLMs.txt" }
  },
  {
    "tool_name": "summarize_file",
    "args": { "filename": "006_Agent Loop with Function Calling.txt" }
  },
  {
    "tool_name": "move_file",
    "args": { "filename": "000_Prompting for Agents -GAIL.txt", "target_folder": "Organized_Summaries" }
  },
  {
    "tool_name": "move_file",
    "args": { "filename": "001_PArse_t



## 🧩 Why Is the Output Inside Triple Backticks?

This:

````python
"```action\n"
"[\n"
"  {...},\n"
"  {...}\n"
"]\n"
"```\n"
````

...is a **Markdown code block**, and it's used **on purpose** to:

### ✅ 1. Make the Output **Easy to Parse**

By putting the JSON inside:

````markdown
```action
[
  { "tool_name": ..., "args": {...} },
  ...
]
```
````

…we can use **regex** to reliably extract it later:

````python
pattern = r"```action\s*(.*?)```"
````

This ensures we can **locate and isolate** the JSON from other surrounding LLM content (like extra commentary, formatting, etc).

---

### ✅ 2. Prevent the LLM from Adding Confusing Output

LLMs sometimes add:

* Explanations: "Here’s your requested JSON:"
* Markdown formatting
* Apologies or extra text

Putting the plan in a ` ```action ` block tells the LLM:

> “Only include tool instructions in *this* block. Everything else goes outside.”

---

### ✅ 3. Allow Multiple Blocks in One Output

You might later use:

* ` ```query ` for a search
* ` ```plan ` for high-level strategy
* ` ```action ` for tool execution

This lets you extract **only the part you care about**, using tags.

---

## 🔍 Why the Word `"action"`?

It's a **custom tag**.

You could use any tag (like `"json"`, `"plan"`, `"result"`), but by calling it `"action"`, you're signaling:

> “This block contains a list of tool-based steps the agent should take.”

---

## 🧠 Summary

| Purpose                          | Benefit                              |
| -------------------------------- | ------------------------------------ |
| ✅ Wrap in \`\`\`action           | Lets you extract only the tool plan  |
| 🧠 Use markdown code block       | Keeps JSON clean and separate        |
| 🧹 Parse cleanly with regex      | No guessing or extra formatting      |
| 🧱 Supports multi-block messages | Easy to separate structured sections |






The ` ```action ` block acts like a **labelled container** or **named index**, so that later in your agent pipeline you can say:

> “Hey, go find the part of the message that contains the **action plan** — and only that part.”

---

## 🔖 Analogy

Think of the LLM’s output as a full document. Inside it, you're marking sections like this:

````
Here’s what I want to do:

```action
[ { "tool_name": ..., "args": {...} } ]
````

Here’s why I’m doing it:

```reasoning
I want to group similar files together to help the user find them faster.
```

````

Now you can write extractors like:

```python
extract_markdown_block(response, tag="action")
extract_markdown_block(response, tag="reasoning")
````

Each tag gives you **structured access** to different parts of the LLM’s reasoning or output.

---

## ✅ Benefits

| Concept                                    | Benefit                                               |
| ------------------------------------------ | ----------------------------------------------------- |
| 🔖 Tags like `action`, `reasoning`, `plan` | Create a schema for multi-step agent logic            |
| 🧼 Markdown code block                     | Keeps JSON clean and parseable                        |
| 📍 Acts like a labeled container           | You can always find it later with a regex             |
| 🧠 Reliable LLM parsing                    | Avoids accidental formatting errors, stray text, etc. |





## 🧰 `"Available tools"` Section — Why It's Included

This part:

```python
"Available tools:\n"
"- summarize_file: takes {\"filename\": str}\n"
"- move_file: takes {\"filename\": str, \"target_folder\": str}"
```

...is a **reference guide** for the LLM. You're giving it a **tool schema** — like API documentation — so it knows:

* What tools it can choose from
* What arguments are required for each tool
* The correct structure and spelling for the tool names and arguments

---

## 🤖 Why Does the LLM Need This?

LLMs are good at language but **don’t know what tools your agent supports** unless you tell them.

So this section does two things:

| Purpose                         | How                                                         |
| ------------------------------- | ----------------------------------------------------------- |
| 🛠️ Defines the “menu” of tools | Gives the names and argument shapes                         |
| 🔐 Prevents hallucination       | Keeps LLM from inventing tools like `"summarize_and_email"` |

You’re saying:

> “Hey LLM, you can ONLY use `summarize_file` and `move_file`, and here’s exactly how they work.”

This guides the LLM to produce output like:

```json
{
  "tool_name": "move_file",
  "args": {
    "filename": "001.txt",
    "target_folder": "Summaries"
  }
}
```

---

## 🧠 It's Like an API Spec

Think of it like this:

* `"Available tools"` = **OpenAPI schema**
* `"tool_name"` + `"args"` = **API call**
* LLM = **caller**
* Your agent = **executor**

So you're giving it a **safe, structured list** to work from.

---

## 🔍 Could You Leave It Out?

Technically yes, but:

* LLM might guess wrong tool names
* Or add invalid arguments
* Or format the JSON incorrectly

💡 Including `"Available tools"` boosts **reliability** and **correct formatting** by a lot — especially when the tool list grows.

---

## ✅ Summary

| Element                  | Purpose                                  |
| ------------------------ | ---------------------------------------- |
| `"action"` tag           | Marks the output section to extract      |
| `"Available tools"`      | Informs the LLM how to respond correctly |
| `"tool_name"` + `"args"` | Defines a single action to take          |






> 🤖 **LLMs are powerful at language, but imprecise by default. Agents require precision.**

---

## 🔍 Why Parsing Is Hard Without Structure

LLMs are trained to write like this:

> "Sure! To summarize `lecture_01.txt`, I suggest moving it to the folder `Summaries`. Let me know if you'd like help renaming it."

That’s natural language — great for humans, **terrible for machines**.

Now imagine trying to extract:

* What file to summarize
* What folder to move it to
* What order those should happen in

🧠 You’d need complex NLP parsing logic… and still risk breaking.

---

## ✅ Why the Structured Format Fixes That

By enforcing:

```json
{
  "tool_name": "move_file",
  "args": {
    "filename": "lecture_01.txt",
    "target_folder": "Summaries"
  }
}
```

...inside a markdown block like:

````
```action
[ ... ]
````

You’re telling the LLM:

- “Don’t talk.”
- “Don’t explain.”
- “Just give me machine-readable instructions.”

That’s **exactly** what enables reliable, automated parsing.

---

## 💥 Without the `"Available tools"` List...

The LLM might return:
- `"moveFile"` instead of `"move_file"`
- `"folder": "Summaries"` instead of `"target_folder"`
- Or invent a tool like `"summarize_and_move"`

Parsing would fail, or your agent might break or do the wrong thing.

---

## 🧠 So Yes, You're Spot On:

| Design Element | Why It Matters |
|----------------|----------------|
| ✅ Parsing | Lets your agent *act* on the LLM’s output |
| 🚫 English prose | Looks pretty, but not actionable |
| ✅ Structured JSON in markdown | Cleanly extractable and reliable |
| ✅ Explicit tool schema | Prevents LLM hallucinations or misuse |
| 🚫 Leaving it to the LLM’s guess | Breaks reliability and makes agents flaky |





## 🧠 Step-by-Step: What Each Function Does

---

### 1. ✅ `extract_markdown_block(text, tag="action")`

This function:

* Searches the LLM response for a **markdown block** like:

  ````
  ```action
  { ... JSON ... }
  ````

  ```
  ```
* It extracts only the part between the backticks, using the `tag` (`action`) to target the correct block.

🔍 **Why?** LLMs often wrap structured data in markdown so you can parse it out cleanly without extra fluff.

---

### 2. ✅ `parse_multi_action_plan(llm_response)`

This function:

* Uses `extract_markdown_block()` to isolate the structured block.
* Then runs `json.loads()` to convert it from **JSON text → Python data structure** (a list of dicts).

📥 Input:
LLM output (as raw string)

📤 Output:

```python
[
  {"tool_name": "summarize_file", "args": {"filename": "file1.txt"}},
  {"tool_name": "move_file", "args": {"filename": "file1.txt", "target_folder": "Summaries"}}
]
```

---

### 3. ✅ `simulate_tool(tool_name, args)`

This is your **agent's executor.**

Right now it's just simulating real behavior:

| Tool               | Simulated Action                                |
| ------------------ | ----------------------------------------------- |
| `"summarize_file"` | Prints `📝 Summarizing: filename.txt`           |
| `"move_file"`      | Prints `📂 Moving 'filename' → 'target_folder'` |
| unknown            | Prints warning: unknown tool                    |

Later, you could replace these print statements with:

* Real summarization using OpenAI
* Real file moving via `shutil.move()`
* Web scraping, emailing, database calls — anything!

---

### 4. ✅ Looping Through the Plan

This is the **agent loop**:

```python
for step in plan:
    simulate_tool(step["tool_name"], step["args"])
```

Each step in the plan is:

```python
{
  "tool_name": "move_file",
  "args": {
    "filename": "lecture_01.txt",
    "target_folder": "Memory"
  }
}
```

So the loop says:

> “Tool = `move_file`, args = `{filename: ..., target_folder: ...}` — now go run that.”

---

## 🧠 Agent Logic Summary

```python
LLM ➜ markdown block ➜ parsed JSON ➜ tool name + args ➜ tool execution
```

The LLM **thinks**.
Your Python code **does**.



**Agents can get expensive surprisingly fast**, especially when:

---

## ⚠️ LLM Agents Cost Risks

| Risk                                     | What It Means                                                       |
| ---------------------------------------- | ------------------------------------------------------------------- |
| 🧾 **Long prompts**                      | Big file lists, long context, detailed instructions                 |
| 📤 **Large outputs**                     | Structured responses (JSON, plans, chains) can be verbose           |
| 🔁 **Multiple LLM calls per agent loop** | If your agent plans, checks, reasons, executes… that adds up fast   |
| 🪜 **Recursive reasoning**               | Agents calling agents (e.g., one LLM interprets another's response) |
| 🧠 **Stateful conversations**            | Chat history gets long = big context every time                     |

---

## 📊 Real Token Cost Breakdown (Estimates)

| Action                                   | Token Cost                 |
| ---------------------------------------- | -------------------------- |
| 10 filenames listed in prompt            | \~150–200 tokens           |
| Structured plan per file                 | \~40–100 tokens per action |
| 10 files with 2 actions each             | \~800–2000+ tokens         |
| One agent loop with planning + execution | \~2–4k tokens total        |
| One detailed summarization per file      | 500–1500 tokens each       |

So if you’re summarizing + organizing 10–20 files in a planner loop…
👉 You're easily at **20,000–40,000 tokens** just for one session.

---

## 💡 Key Insight

> **Simple apps are cheap. Agents are powerful — but power has a price.**

Agents bring more:

* Flexibility
* Reasoning
* Control flow
* Autonomy
  …but those advantages come at the cost of **longer prompts**, **richer output**, and **multi-turn processing**.

---

## ✅ Cost Management Tips

| Strategy                                          | Result                             |
| ------------------------------------------------- | ---------------------------------- |
| 🔹 Use `gpt-3.5-turbo` for general planning       | 90% of the time it's “good enough” |
| 🔹 Log token usage per run (`usage.total_tokens`) | Monitor & alert on large jobs      |
| 🔹 Save and reuse plans                           | Only re-call LLM if inputs change  |
| 🔹 Use local code for filtering, clustering       | Let LLM do only what Python can't  |
| 🔹 Use batching or a queueing system              | Avoid massive prompts in one go    |


## 🤔 So Why Was It More Expensive?

Likely because of:

### 1. **Prompt Size**

* If you have many files, the `file_list_str` is long.
* More input tokens = higher cost.

### 2. **LLM Output Size**

* The model returns a JSON list of actions, which can be long if there are lots of files.
* More output tokens = higher cost.

### 3. **Model Choice**

* Even `gpt-4o-mini` is more expensive than `gpt-3.5-turbo`, though it's faster and smarter.

---

## ✅ Ways to Save on Cost

| Strategy                     | How                                                                      |
| ---------------------------- | ------------------------------------------------------------------------ |
| 🧹 **Summarize fewer files** | Only pass a subset of files                                              |
| 🔄 **Batch the LLM calls**   | Break files into chunks of 5–10                                          |
| ⚡ **Use a cheaper model**    | Switch to `gpt-3.5-turbo` if accuracy is acceptable                      |
| 🧠 **Preprocess locally**    | Use local Python (e.g., keywords) to cluster/group files before LLM call |
| 📉 **Log & cache results**   | Save plans so you don’t re-query the LLM for the same files repeatedly   |

---

## ⚙️ Optional Optimization

You could break this into a batching loop like:

```python
from math import ceil

batch_size = 5
batches = [file_list[i:i + batch_size] for i in range(0, len(file_list), batch_size)]

all_steps = []

for batch in batches:
    messages = build_multi_action_prompt(batch)
    llm_output = generate_response(messages)
    plan = parse_multi_action_plan(llm_output)
    all_steps.extend(plan)
```

This breaks up big prompts and avoids a huge single LLM call — which helps with:

* Cost
* Speed
* Token limit risks

---

## 🧠 Bottom Line

* ✅ You're doing the right thing so far (1 call total).
* 🧠 As scale grows, batching and optimization matter.
* 💰 You’re building cost-aware, production-capable agents — huge milestone!



# Prompt + Validation in the Real World

In production you *do both*: prompt the model to emit structured JSON **and** validate what you get. Prompts **steer**; validation **guarantees**. Models are probabilistic, can drift, and occasionally ignore formatting—so you need a contract (schema) that your Python enforces before any tool runs.

Why this combo works:

* **Reliability:** Prompting raises the success rate; schema validation catches the misses.
* **Safety:** You never execute unknown tools or bad args.
* **Maintainability:** Clear errors (from validation) feed back into retries and prompt tuning.
* **Determinism:** Structured data lets you route, log, and chain actions predictably.

Here’s a tiny drop-in hardening for your multi-action plan:

````python
import json, re, os
from typing import List, Union, Literal
from pydantic import BaseModel, ValidationError

# 1) Slightly tougher extractor (accepts ```action or ```action json)
def extract_markdown_block(text: str, tag: str = "action") -> str:
    pattern = re.compile(rf"```{tag}(?:\s+json)?\s*(.*?)```", re.DOTALL | re.IGNORECASE)
    m = pattern.search(text)
    if not m:
        raise ValueError(f"Missing markdown block with tag '{tag}'")
    return m.group(1).strip()

# 2) Schemas (the contract)
class SummarizeArgs(BaseModel):
    filename: str

class MoveArgs(BaseModel):
    filename: str
    target_folder: str

class SummarizeAction(BaseModel):
    tool_name: Literal["summarize_file"]
    args: SummarizeArgs

class MoveAction(BaseModel):
    tool_name: Literal["move_file"]
    args: MoveArgs

PlanItem = Union[SummarizeAction, MoveAction]

# 3) Parse + validate + semantic checks
def parse_multi_action_plan(llm_response: str, allowed_files: set[str], max_actions: int = 50) -> List[dict]:
    block = extract_markdown_block(llm_response, tag="action")
    try:
        raw = json.loads(block)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON in action block: {e}") from e

    if not isinstance(raw, list):
        raise ValueError("Expected a JSON list of actions.")

    if len(raw) > max_actions:
        raise ValueError(f"Plan too long (> {max_actions} actions).")

    validated: List[dict] = []
    for i, item in enumerate(raw, 1):
        try:
            model = SummarizeAction.model_validate(item) if item.get("tool_name") == "summarize_file" \
                    else MoveAction.model_validate(item) if item.get("tool_name") == "move_file" \
                    else None
            if model is None:
                raise ValueError(f"Unknown tool '{item.get('tool_name')}'.")
        except ValidationError as ve:
            raise ValueError(f"Schema error at action #{i}: {ve.errors()}") from ve

        # semantic checks
        fn = os.path.basename(model.args.filename)
        if fn not in allowed_files:
            raise ValueError(f"Unknown or disallowed filename at action #{i}: {fn}")

        validated.append(model.model_dump())

    return validated
````

**Usage pattern in prod:**

1. Prompt for JSON (as you do).
2. Extract → `json.loads` → **Pydantic validate** (+ semantic checks).
3. On failure: **retry once** with stricter instructions/lower temperature; else surface a clean error.

That’s the pragmatic, reliable way to turn LLM plans into safe, executable agent actions.


##**Prompt-only formatting is brittle.**

In production, you *constrain and validate* every time.

## The reliability ladder (strongest → weakest)

1. **Native tool/function calling** (API returns `{name, arguments}`): no free-text to parse; still validate.
2. **JSON mode + schema / constrained decoding**: model is steered to emit valid JSON of a known shape; still validate.
3. **Pydantic (or JSON Schema) validation**: enforce types, required fields, enums, ranges; reject bad outputs.
4. **Fenced JSON + parser** (` ```action { ... } ``` `): workable, but needs schema validation + retry.
5. **Prompt-only “please format like …”**: OK for demos; too flaky for automation.

## Why prompt-only loses

* Models drift; prompts evolve; occasional hallucinated prose sneaks in.
* Minor changes (temperature, context length, upstream model updates) break formatting.
* No types, no bounds, no whitelist → higher risk to tools with side effects.

## What to ship

* **Constrain → Parse → Validate → Retry (≤2) → Fallback/log.**
* Keep a **schema version** in outputs; add **semantic checks** (e.g., `b != 0`).
* Always log validation failures to improve prompts/schemas.

## When prompt-only is acceptable

* Prototypes, internal tools with a human-in-the-loop, or low-stakes outputs.

**Bottom line:** Use the strongest mechanism available (tool calling or JSON mode), and **always** layer Pydantic/Schema validation on top. Prompts help—but contracts and validation make agents reliable.


## **Validate Forever.**
Treat *all* LLM output as untrusted input—during dev **and** in production. Even with JSON mode or tool-calling you still validate, because models drift, prompts evolve, and edge cases show up in the wild.

## What to validate (two layers)

1. **Structural (syntactic)**

* Valid JSON / matches schema (Pydantic/JSON Schema)
* Required keys present, correct types
* Enums/allowlists for `tool_name`
* Length limits (e.g., summary ≤ 120 words), regexes, etc.

2. **Semantic (business rules)**

* `divide`: `b != 0`
* File ops: paths stay in allowed directory
* URLs on an allowlist; no network where forbidden
* Budget/date bounds, idempotency tokens, etc.

## Runtime policy (keep this in prod)

* **Validate → execute**; on failure: **retry once** (maybe twice) with stricter instructions/lower temperature.
* If still invalid: **graceful fallback** (return a safe error or a simpler baseline behavior).
* **Always log** invalid outputs, retry counts, and tool failures; set alerts if rates spike.

## Observability you want

* Metrics: `llm_parse_invalid_rate`, `validation_errors_by_field`, `tool_call_success_rate`, latency, token/cost.
* Attach `schema_version` to every response so you can evolve safely.
* Circuit breaker or feature flag to tighten/relax validation without redeploy.

## Tiny pattern (prod-safe wrapper)

```python
from pydantic import BaseModel, Field, ValidationError
from typing import Literal, Dict

class Action(BaseModel):
    schema_version: int = Field(1, const=True)
    tool_name: Literal["multiply", "divide", "search"]
    args: Dict

def validate_and_dispatch(raw_text: str):
    try:
        block = extract_markdown_block(raw_text, "action")
        action = Action.model_validate_json(block)  # structural
        # semantic checks
        if action.tool_name == "divide" and action.args.get("b") == 0:
            raise ValueError("Division by zero.")
        return dispatch(action.tool_name, action.args)
    except (ValidationError, ValueError) as e:
        # log error, optionally retry once with stricter prompt/JSON mode
        return {"error": f"Validation failed: {e}"}
```

## Bottom line

* **Validation is not a dev-only safety net—it’s the production contract.**
* Keep it **always-on**, layer structural + semantic checks, cap retries, and monitor. This is what turns LLM glue code into a reliable agent system.
